In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


날씨와 농산물 가격은 어떤 상관 관계가 있는지 예측한다.  
기상 정보: 기상 자료 개방 포털(https://data.kma.go.kr), 농산물 가격 정보: 농산물 유통 정보(https://www.kamis.or.kr)

기상 정보: 평균 기온(avgTemp), 최저 기온(minTemp), 최고 기온(maxTemp), 강수량(rainFall)  
가격 정보: 배추 가격(avgPrice)

In [2]:
# price_data.csv 파일을 읽어서 데이터프레임으로 저장한다.
# csv 파일의 첫 줄에 컬럼 이름(헤더)이 없을 경우 아래와 같이 header 속성을 지정한 후 읽어서 컬럼 이름을 붙여준다.
# price_data = pd.read_csv('./data/price_data.csv', header=None)
# price_data.columns = ['year',  'avgTemp', 'minTemp', 'maxTemp', 'rainFall', 'avgPrice']
price_data = pd.read_csv('./data/price_data.csv')
price_data

year  avgTemp  minTemp  maxTemp  rainFall  avgPrice
0     20100101     -4.9    -11.0      0.9       0.0      2123
1     20100102     -3.1     -5.5      5.5       0.8      2123
2     20100103     -2.9     -6.9      1.4       0.0      2123
3     20100104     -1.8     -5.1      2.2       5.9      2020
4     20100105     -5.2     -8.7     -1.8       0.7      2060
...        ...      ...      ...      ...       ...       ...
2917  20171227     -3.9     -8.0      0.7       0.0      2865
2918  20171228     -1.5     -6.9      3.7       0.0      2884
2919  20171229      2.9     -2.1      8.0       0.0      2901
2920  20171230      2.9     -1.6      7.1       0.6      2901
2921  20171231      2.1     -2.0      5.8       0.4      2901

[2922 rows x 6 columns]

In [3]:
# 데이터프레임에 저장된 데이터를 텐서플로우에서 처리하기에 적합하도록 넘파이 배열 형태로 변환한다.
data = np.array(price_data, dtype=np.float32)
print(type(data))
data

<class 'numpy.ndarray'>


array([[ 2.0100100e+07, -4.9000001e+00, -1.1000000e+01,  8.9999998e-01,
         0.0000000e+00,  2.1230000e+03],
       [ 2.0100102e+07, -3.0999999e+00, -5.5000000e+00,  5.5000000e+00,
         8.0000001e-01,  2.1230000e+03],
       [ 2.0100104e+07, -2.9000001e+00, -6.9000001e+00,  1.4000000e+00,
         0.0000000e+00,  2.1230000e+03],
       ...,
       [ 2.0171228e+07,  2.9000001e+00, -2.0999999e+00,  8.0000000e+00,
         0.0000000e+00,  2.9010000e+03],
       [ 2.0171230e+07,  2.9000001e+00, -1.6000000e+00,  7.0999999e+00,
         6.0000002e-01,  2.9010000e+03],
       [ 2.0171232e+07,  2.0999999e+00, -2.0000000e+00,  5.8000002e+00,
         4.0000001e-01,  2.9010000e+03]], dtype=float32)

In [4]:
# 넘파이 배열에서 변화 요인(피쳐) 데이터(평균 기온, 최저 기온, 최고 기온, 강수량)로 사용할 데이터를 뽑아낸다.
xData = data[:, 1:5]
print(type(xData))
xData

<class 'numpy.ndarray'>


array([[ -4.9, -11. ,   0.9,   0. ],
       [ -3.1,  -5.5,   5.5,   0.8],
       [ -2.9,  -6.9,   1.4,   0. ],
       ...,
       [  2.9,  -2.1,   8. ,   0. ],
       [  2.9,  -1.6,   7.1,   0.6],
       [  2.1,  -2. ,   5.8,   0.4]], dtype=float32)

In [5]:
# 넘파이 배열에서 결과(타겟) 데이터(평균 가격)로 사용할 데이터를 뽑아낸다.
# yData = data[:, -1] # 인덱싱으로 뽑아내면 1차원 형태로 데이터를 뽑아낸다.
# 변화 요인이 2차원이므로 결과도 2차원 형태로 뽑아내야 한다.
# print(type(yData))
# yData.reshape(-1, 1) # 1차원 형태로 뽑아냈다면 reshape() 메소드로 2차원으로 변형한다.
# yData = data[:, 5:6] # 슬라이싱으로 뽑아내면 2차원 형태의 데이터로 뽑아낸다.
yData = data[:, [-1]] # 인덱싱으로 뽑아낸 결과를 리스트화 시켜도 2차원 형태의 데이터로 뽑아낸다.
yData

array([[2123.],
       [2123.],
       [2123.],
       ...,
       [2901.],
       [2901.],
       [2901.]], dtype=float32)

In [6]:
# 뽑아낸 데이터를 텐서플로우에서 처리하기 위해서 placeholder를 만든다.
X = tf.placeholder(dtype=tf.float32, shape=[None, 4]) # 변화 요인을 기억할 placeholder를 만든다. 학습 데이터
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1]) # 결과를 기억할 placeholder를 만든다. 실제값

In [7]:
# 다변인 선형 회귀 모델의 기울기(가중치)와 y절편(바이어스)을 임의의 값으로 초기화 한다.
a = tf.Variable(tf.random_uniform([4, 1]), dtype=tf.float32) # 기울기, 내적을 하기 위해서 4행 1열의 난수를 발생시킨다.
b = tf.Variable(tf.random_uniform([1]), dtype=tf.float32) # y절편
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('가중치\n', sess.run(a), sep='')
print('y절편\n', sess.run(b), sep='')

가중치
[[0.56290376]
 [0.8843403 ]
 [0.7419512 ]
 [0.51994276]]
y절편
[0.51350594]


In [8]:
# 행렬의 곱셈(내적) 연산을 이용해서 다변인 선형 회귀 모델의 가설로 사용할 식을 만든다. => 예측값을 계산하는 식
y = tf.matmul(X, a) + b # 예측값
# 오차(비용) 함수를 만든다. => 평균 제곱법 => 예측값(y)과 실제값(Y)의 편차에 제곱에 대한 평균
loss = tf.reduce_mean(tf.square(y - Y))
# 경사 하강법 알고리즘을 사용해서 오차 함수의 결과를 최소로 하는 기울기와 y절편을 찾는다.
gradient_descent = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

In [9]:
# 학습 시킨다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(100001):
    _, loss_, y_ = sess.run([gradient_descent, loss, y], feed_dict={X: xData, Y: yData})
    if epoch % 5000 == 0:
        # print('epoch: %6d, loss: %12.3f, price: %8.2f' % (epoch, loss_, y_[0]))
        print('epoch: {:6d}, loss: {:12.3f}, price: {:8.2f}'.format(epoch, loss_, y_[0][0]))

epoch:      0, loss: 12440061.000, price:   -12.68
epoch:   5000, loss:  1965923.125, price:  2649.81
epoch:  10000, loss:  1873624.000, price:  2720.14
epoch:  15000, loss:  1842593.500, price:  2771.77
epoch:  20000, loss:  1831585.750, price:  2805.99
epoch:  25000, loss:  1827611.625, price:  2827.66
epoch:  30000, loss:  1826169.375, price:  2841.08
epoch:  35000, loss:  1825643.875, price:  2849.28
epoch:  40000, loss:  1825453.500, price:  2854.27
epoch:  45000, loss:  1825383.750, price:  2857.29
epoch:  50000, loss:  1825358.125, price:  2859.13
epoch:  55000, loss:  1825349.000, price:  2860.22
epoch:  60000, loss:  1825345.875, price:  2860.89
epoch:  65000, loss:  1825345.000, price:  2861.29
epoch:  70000, loss:  1825344.500, price:  2861.52
epoch:  75000, loss:  1825344.125, price:  2861.69
epoch:  80000, loss:  1825343.750, price:  2861.73
epoch:  85000, loss:  1825343.750, price:  2861.74
epoch:  90000, loss:  1825344.000, price:  2861.74
epoch:  95000, loss:  1825344.0

In [10]:
# 테스트
data = [[5.2, -7.5, 10.1, 15.0]]
result = sess.run(y, feed_dict={X: data})
print(result)
print('평균 온도: {:4.1f}, 최저 온도: {:4.1f}, 최고 온도, {:4.1f}, 강수량: {:4.1f}일 때 '\
      '배추 가격은 {:7,.2f}원 입니다.'.format(data[0][0], data[0][1], data[0][2], data[0][3], result[0][0]))

[[2802.2605]]
평균 온도:  5.2, 최저 온도: -7.5, 최고 온도, 10.1, 강수량: 15.0일 때 배추 가격은 2,802.26원 입니다.


In [11]:
# 학습 완료된 예측 모델을 디스크에 저장한다.
# Saver() 객체로 텐서플로우에서 학습된 모델을 디스크에 저장 또는 불러올 때 사용하는 객체를 생성한다.
saver = tf.train.Saver()
saver.save(sess, './model/baechu.ckpt')
print('학습된 모델을 저장했습니다.')

학습된 모델을 저장했습니다.
